# Traits and Generics

## Traits

* Traits are Rust’s take on interfaces or abstract base classes

In [9]:
trait Printable {
    fn print(&self);
}

### Implementing Traits

In [11]:
struct Number {
    value: i32,
}

impl Printable for Number {
    fn print(&self) {
        println!("Number: {}", self.value);
    }
}

impl Printable for f64 {
    fn print(&self) {
        println!("Float: {}", self);
    }
}

fn main() {
    let a = Number { value: 10 };
    let b = 5.5;

    a.print();
    b.print();
}

main();

Number: 10
Float: 5.5


In [25]:
trait Shape {
    fn move_by(&mut self, dx: i32, dy: i32);
    fn area(&self) -> f64;
}

#[derive(Debug)]
struct Point {
    x: i32,
    y: i32
}

#[derive(Debug)]
struct Circle {
    position: Point,
    radius: u32
}

#[derive(Debug)]
struct Rectangle {
    position: Point,
    width: u32,
    height: u32
}

impl Shape for Circle {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        std::f64::consts::PI * (self.radius as f64) * (self.radius as f64)
    }
}

impl Shape for Rectangle {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        (self.width * self.height) as f64
    }
}

### Default Trait Behavior

In [26]:
trait Drawable {
    fn draw(&self) {
        println!("Drawing the shape...");
    }
}

impl Drawable for Circle {
    fn draw(&self) {
        println!("Drawing the circle at {:?} with radius: {}", self.position, self.radius);
    }
}

impl Drawable for Rectangle {
    fn draw(&self) {
          println!("Drawing the rectangle at {:?} with width: {} and height: {}", self.position, self.width, self.height);
  }
}

### Trait Bounds

#### `impl Trait` Syntax

In [27]:
fn draw_shape(shape: &impl Drawable) {
    shape.draw()
}

let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
draw_shape(&circle);

let rect = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};
draw_shape(&rect);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


#### Trait Bound Syntax

In [29]:
fn draw_shape<T: Drawable>(shape: &T) {
    shape.draw()
}

draw_shape(&circle);
draw_shape(&rect);

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5


#### Many trait bounds

In [32]:
fn use_shape(shape: &mut (impl Shape + Drawable)) {
    shape.move_by(10, 15);
    shape.draw();
}

In [34]:
fn use_shape<T: Shape + Drawable>(shape: &mut T) {
    shape.move_by(10, 15);
    shape.draw();
}

#### Clearer Trait Bounds with where Clauses

In [35]:
fn some_function<T: Printable + Clone, U: Drawable + Clone>(t: &T, u: &U) {
    t.print();
    u.draw();
}

In [36]:
fn some_function<T, U>(t: &T, u: &U)
where 
    T: Printable + Clone,
    U: Drawable + Clone,
{
    t.print();
    u.draw();
}

### Returning Types that Implement Traits

In [40]:
fn create_drawable() -> Box<dyn Drawable> {
    Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5})
}

let shape: Box<dyn Drawable> = create_drawable();
shape.draw();

Drawing the circle at Point { x: 10, y: 20 } with radius: 5


### Self in Traits

* A trait can use the keyword `Self` as a type

In [2]:
trait Cloneable {
    fn clone(&self) -> Self; // the type of x.clone() is the same as the type of x, whatever that might be
}

### Trait Objects

In [ ]:
let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};

let shape: dyn Shape = circle; // ERROR: Shape does not have a constant size

#### References to Trait

In [52]:
fn main() {
    let circle = Circle{position: Point{x: 10, y: 20}, radius: 5};

    let shape: & dyn Shape = &circle; // OK - shape is trait object

    println!("Shape area: {:.2}", shape.area());
}

main();

Shape area: 78.54


* A reference to trait type, like `shape` is called **trait object**. Like any other reference, a trait object points to some value, it has a lifetime, and it can be either mut or shared.
* In memory, a trait object is a *fat pointer* consisting of a pointer to the value, plus a pointer to a table representing that value’s type (virtual table).



#### Boxes to Trait

In [55]:
let box_circle: Box<Circle> = Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5});

let shape: Box<dyn Shape> = box_circle; // OK - Box<dyn Shape> is trait object

println!("Shape area: {:.2}", shape.area());

Shape area: 78.54


## Dynamic Polymorphism in Rust

In [71]:
#[derive(Debug)]
struct Square {
    position: Point,
    side: u32
}   

impl Shape for Square {

    fn move_by(&mut self, dx: i32, dy: i32) {
        self.position.x += dx;
        self.position.y += dy;
    }

    fn area(&self) -> f64 {
        (self.side * self.side) as f64
    }
}

impl Drawable for Square {
    fn draw(&self) {
        println!("Drawing the square at {:?} with side: {}", self.position, self.side);
    }
}

fn main() {
    let mut shapes: Vec<Box<dyn Shape>> = vec![
        Box::new(Circle{position: Point{x: 10, y: 20}, radius: 5}),
        Box::new(Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5}),
        Box::new(Square{position: Point{x: 70, y: 80}, side: 7})
    ];

    for shape in shapes.iter_mut() {
        shape.move_by(5, 5);
        println!("Shape area: {:.2}", shape.area());
    }
}

main();

Shape area: 78.54
Shape area: 50.00
Shape area: 49.00


### Combining Traits

In [72]:
trait DrawableShape: Shape + Drawable {} // Trait that combines Shape and Drawable

impl<T> DrawableShape for T where T: Shape + Drawable {} // Implement DrawableShape for any type that implements Shape and Drawable

fn use_shape(shape: &mut dyn DrawableShape) {
    shape.draw();
    shape.move_by(10, 15);
    shape.draw();
}

fn main() {
    let mut circle = Circle{position: Point{x: 10, y: 20}, radius: 5};
    use_shape(&mut circle);

    let mut rect = Rectangle{position: Point{x: 40, y: 50}, width: 10, height: 5};
    use_shape(&mut rect);
}

main();

Drawing the circle at Point { x: 10, y: 20 } with radius: 5
Drawing the circle at Point { x: 20, y: 35 } with radius: 5
Drawing the rectangle at Point { x: 40, y: 50 } with width: 10 and height: 5
Drawing the rectangle at Point { x: 50, y: 65 } with width: 10 and height: 5


## Associated Types

In [15]:
trait Iterator {
    type Item; // associated type
    fn next(&mut self) -> Option<Self::Item>;
}

struct Counter {
    value: u32,
    sentinel: u32
}

impl Counter {
    fn new(sentinel: u32) -> Counter {
        Counter{value: 0, sentinel}
    }
}

impl Iterator for Counter {
    type Item = u32;

    fn next(&mut self) -> Option<Self::Item> {
        if self.value < self.sentinel {
            let result = self.value;
            self.value += 1;
            Some(result)
        } else {
            None
        }
    }
}

fn collect_into_vector<I: Iterator>(mut iter: I) -> Vec<I::Item>
{
    let mut results = Vec::new();

    while let Some(value) = iter.next() {
        results.push(value);
    }

    results
}

fn main() {
    let mut counter = Counter::new(5);

    while let Some(value) = counter.next() {
        println!("Counter value: {}", value);
    }

    let numbers = collect_into_vector(Counter::new(5));
    println!("Collected numbers: {:?}", numbers);
}

main();

Counter value: 0
Counter value: 1
Counter value: 2
Counter value: 3
Counter value: 4
Collected numbers: [0, 1, 2, 3, 4]


## Generics

### Generic Function

In [18]:
fn min<T: Ord>(a: T, b: T) -> T {
    if a < b {
        a
    } else {
        b
    }
}

In [19]:
min(1, 2)

1

In [20]:
min("hello", "world")

"hello"

### Generic Structure

In [73]:
struct Stack<T> {
    items: Vec<T>
}

impl<T> Stack<T> {
    fn push(&mut self, item: T) {
        self.items.push(item);
    }

    fn pop(&mut self) -> Option<T> {
        self.items.pop()
    }
}

fn main() {
    let mut stack = Stack{items: Vec::new()};
    stack.push(1);
    stack.push(2);
    stack.push(3);

    while let Some(item) = stack.pop() {
        println!("Popped: {}", item);
    }
}

main();

Popped: 3
Popped: 2
Popped: 1
